In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer


# In[2]:


filmes = pd.read_csv("movies_metadata.csv")
creditos = pd.read_csv("credits.csv")

#Usando as palavras chaves no dataset keywords

palavraschaves = pd.read_csv("keywords.csv")


# In[3]:


filmes.head()


# In[4]:


filmes = filmes[["id", "title", "genres","vote_average", "vote_count"]]


# In[5]:


#Função de limpar os ID's

def limpa_id(x):
    
    
    try: 
        
        return int(x)
    
    except:
        
        
        return np.nan
    
    


# In[6]:


filmes.head()


# In[7]:


filmes['id'] = filmes['id'].apply(limpa_id)
filmes = filmes[filmes['id'].notnull()]


# In[8]:


filmes['id'] = filmes['id'].astype('int')
palavraschaves['id'] = palavraschaves['id']
creditos['id'] = creditos['id'].astype('int')


# In[9]:


#Juntando através do id

filmes1 = filmes.merge(creditos, on = "id")
filmes2 = filmes1.merge(palavraschaves, on = "id")


# In[10]:


filmes.head()


# In[11]:


filmes1.head()


# In[12]:


filmes2.head()


# In[13]:


creditos.head()


# In[14]:


# aplicando literal_eval para fazer string -> objetos.

filmes2['genres'] = filmes2['genres'].apply(literal_eval)


# In[15]:


filmes2.head()


# In[17]:


filmes2.info()


# In[18]:


filmes2['cast'] = filmes2['cast'].apply(literal_eval)


# In[19]:


filmes2['crew'] = filmes2['crew'].apply(literal_eval)
filmes2['keywords'] = filmes2['keywords'].apply(literal_eval)


# In[20]:


#pegando o nome dos generos associados
filmes2['crew'] = filmes2['crew'].apply(lambda x: [i['name'].lower() for i in x if i['job'] == 'Director'])


# In[25]:


#juntando com as palavras chaves
df = filmes2


# In[26]:


df.head()


C:\Users\lucas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            46628 non-null  int32  
 1   title         46624 non-null  object 
 2   genres        46628 non-null  object 
 3   vote_average  46624 non-null  float64
 4   vote_count    46624 non-null  float64
 5   cast          46628 non-null  object 
 6   crew          46628 non-null  object 
 7   keywords      46628 non-null  object 
dtypes: float64(2), int32(1), object(5)
memory usage: 3.0+ MB


,id,title,genres,vote_average,vote_count,cast,crew,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",[john lasseter],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",[joe johnston],"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",[howard deutch],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...",[forest whitaker],"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...",[charles shyer],"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [2]:
vote = df[df["vote_count"].notnull()]["vote_count"].astype('float')
avg = df[df["vote_average"].notnull()]["vote_average"].astype('float')
C = avg.mean()
m = vote.quantile(0.50)
qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())]
def imdb_qualified(x):
    v = x["vote_count"]
    R = x["vote_average"]
    return (v/(v+m) * R) + (m/(m+v) * C)
qualified['wr'] = qualified.apply(imdb_qualified, axis = 1)
df = qualified[["id", "title", "genres", "cast", "crew", "keywords"]]
df["genres"] = df["genres"].apply(lambda x: [i["name"].lower() for i in x])
df["cast"] = df["cast"].apply(lambda x: [i["name"].lower() for i in x])
df["keywords"] = df["keywords"].apply(lambda x: [i["name"].lower() for i in x])
df["genres"] = df["genres"].apply(lambda x: x[:3] if len(x)>3 else x)
df["cast"] = df["cast"].apply(lambda x: x[:3] if len(x)>3 else x)
df["keywords"] = df["keywords"].apply(lambda x: x[:3] if len(x)>3 else x)
df["cast"] = df["cast"].apply(lambda x: [i.replace(" ","") for i in x])
df["crew"] = df["crew"].apply(lambda x: [i.replace(" ","") for i in x])
df["keywords"] = df["keywords"].apply(lambda x: [i.replace(" ","") for i in x])
df["genres"] = df["genres"].apply(lambda x: [i.replace(" ","") for i in x])

<ipython-input-2-397498d29902>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['wr'] = qualified.apply(imdb_qualified, axis = 1)
<ipython-input-2-397498d29902>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["genres"] = df["genres"].apply(lambda x: [i["name"].lower() for i in x])
<ipython-input-2-397498d29902>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [3]:
df.head()

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]"
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single..."
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]"


In [4]:
df["metadata"] = df.apply(lambda x : " ".join(x["genres"]) + " "  + " ".join(x["cast"]) + " " + " ".join(x["crew"]) + " " + " ".join(x["keywords"]), axis = 1)


<ipython-input-4-36f122607b46>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["metadata"] = df.apply(lambda x : " ".join(x["genres"]) + " "  + " ".join(x["cast"]) + " " + " ".join(x["crew"]) + " " + " ".join(x["keywords"]), axis = 1)


In [5]:
df.head()

,id,title,genres,cast,crew,keywords,metadata
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]",animation comedy family tomhanks timallen donr...
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]",adventure fantasy family robinwilliams jonatha...
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]",romance comedy waltermatthau jacklemmon ann-ma...
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single...",comedy drama romance whitneyhouston angelabass...
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]",comedy stevemartin dianekeaton martinshort cha...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23374 entries, 0 to 46622
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        23374 non-null  int32 
 1   title     23374 non-null  object
 2   genres    23374 non-null  object
 3   cast      23374 non-null  object
 4   crew      23374 non-null  object
 5   keywords  23374 non-null  object
 6   metadata  23374 non-null  object
dtypes: int32(1), object(6)
memory usage: 1.3+ MB


In [7]:
df_metadata = df.iloc[0:20000, 6]

In [8]:
df_metadata

0        animation comedy family tomhanks timallen donr...
1        adventure fantasy family robinwilliams jonatha...
2        romance comedy waltermatthau jacklemmon ann-ma...
3        comedy drama romance whitneyhouston angelabass...
4        comedy stevemartin dianekeaton martinshort cha...
                               ...                        
35788    romance comedy drama marthahigareda kunobecker...
35789    romance comedy drama marthahigareda kunobecker...
35792    war history romance ranveersingh deepikapaduko...
35793    war history romance ranveersingh deepikapaduko...
35798    action drama thriller jean-claudevandamme brad...
Name: metadata, Length: 20000, dtype: object

In [9]:
#Usando cosseno de similaridade para achar a semelhança entre dois filmes

cv = CountVectorizer(stop_words = 'english')

contador_matrix = cv.fit_transform(df_metadata)

cosseno_sim_matrix = cosine_similarity(contador_matrix)

In [10]:
cosseno_sim_matrix

array([[1.        , 0.09534626, 0.1       , ..., 0.        , 0.        ,
        0.        ],
       [0.09534626, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.1       , 0.        , 1.        , ..., 0.1       , 0.1       ,
        0.        ],
       ...,
       [0.        , 0.        , 0.1       , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.1       , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [11]:
#Mapeando a função 

mapear = pd.Series(df_metadata.index, index = df.iloc[0:20000, 1])
mapear

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Te presento a Laura            35788
Te presento a Laura            35789
Bajirao Mastani                35792
Bajirao Mastani                35793
Swelter                        35798
Length: 20000, dtype: int64

In [12]:
df_metadata

0        animation comedy family tomhanks timallen donr...
1        adventure fantasy family robinwilliams jonatha...
2        romance comedy waltermatthau jacklemmon ann-ma...
3        comedy drama romance whitneyhouston angelabass...
4        comedy stevemartin dianekeaton martinshort cha...
                               ...                        
35788    romance comedy drama marthahigareda kunobecker...
35789    romance comedy drama marthahigareda kunobecker...
35792    war history romance ranveersingh deepikapaduko...
35793    war history romance ranveersingh deepikapaduko...
35798    action drama thriller jean-claudevandamme brad...
Name: metadata, Length: 20000, dtype: object

In [13]:
def sistema(filmes_input):
    
    filme_indexado = mapear[filmes_input]
    
    #obtendo os valores similares
    
    score = list(enumerate (cosseno_sim_matrix[filme_indexado]))
    
    score = sorted(score, key = lambda x: x[1], reverse = True)
    
    # Amostra de 20 filmes
    
    
    score = score[1:20]
    
    indices = [i[0] for i in score]
    
    return (mapear.iloc[indices])

In [14]:
sistema("Jumanji")

title
Where the Wild Things Are                       14455
Mostly Ghostly                                  26717
Tinker Bell and the Lost Treasure               24871
Mostly Ghostly: Have You Met My Ghoulfriend?    30651
The Pagemaster                                    552
City of Ember                                   13109
Zenon: Girl of the 21st Century                 30667
The Water Horse                                 12403
Snow Queen                                      26406
Return to Oz                                     2004
Karlsson on the Roof                            27913
Dragon Nest: Warriors' Dawn                     32574
Santa Claus: The Movie                           2306
The Slipper and the Rose                         3517
Epic                                            21172
The Games Maker                                 30460
The Indian in the Cupboard                         59
The Wizard of Oz                                  906
Aladdin and the King o

In [15]:
def sistema2(filmes_input):
  
  #"Somando os filmes"
  indice = 19000
  df_metadata[indice] = ' '
  for filme in filmes_input:
    df_metadata[indice] += ' ' + df_metadata[mapear[filme]]

  #Usando a similaridade de cosseno
  cv = CountVectorizer(stop_words = 'english')
  contador_matrix = cv.fit_transform(df_metadata)
  cosseno_sim_matrix = cosine_similarity(contador_matrix)

  #Resetando a soma de filmes
  df_metadata[indice] = ' '

  #obtendo os valores similares
  
  score = list(enumerate(cosseno_sim_matrix[indice]))
  score = sorted(score, key = lambda x: x[1], reverse = True)
    
  # Amostra de 20 filmes

  n_filmes = len(filmes_input)
  score = score[(n_filmes+1):20]
  indices = [i[0] for i in score]
  
  #return (score)
  return (mapear.iloc[indices])

In [16]:
sistema2(['Toy Story', "Cars"])

title
Flash of Genius                       13125
Gifted Hands: The Ben Carson Story    19155
James Dean                            28473
Gods                                  28769
Shahid                                30890
Freud: The Secret Passion              7612
Lola Montès                            7706
Napoleon                              12601
Vincere                               13854
Padre Padrone                         14320
Animal Kingdom                        15784
Stavisky                              19212
Ip Man: The Final Fight               21466
Kill Your Darlings                    22082
Son of God                            23192
Summer in February                    25749
Rudy                                    519
dtype: int64